In [1]:
import numpy as np
from tensorflow import keras
from tools import signature as sig
from sklearn.preprocessing import StandardScaler

from ipywidgets import interact, fixed
import ipywidgets as widgets

In [2]:
def signature(keep, n):
    keep.sort()
    (x_train_origin, y_train_origin), (x_test_origin, y_test_origin)= \
    keras.datasets.mnist.load_data(path='all_mnist.npz')

    (x, y), (x_test, y_test) = sig.pretraitement(x_train_origin, y_train_origin, 
                                             x_test_origin, y_test_origin, 
                                             keep=keep, n=n, verbose=False)

    layers = [32, 16]
    model = sig.trained_model(x, y, hidden_layers=layers, verbose=False)

    loss, acc = model.evaluate(x_test, y_test)
    if False:
        print("loss =", loss, ", accuracy =",acc)

    predictions = sig.get_preditions(model, x, keep, verbose=False)
    labels = [keep[np.argmax(l)] for l in y]

    hidden_layers = model.get_hidden_layers_outputs(x)

    # Standardization 
    # A noter que sans cette standardisation on a de plus beau score silhouette
    for i in range(len(hidden_layers)):
        hidden_layers[i] = StandardScaler().fit_transform(hidden_layers[i])

    best_k = sig.get_best_k(hidden_layers, kmax = 10,verbose=False)

    clusters = sig.clustering(hidden_layers, best_k)

    sig.umap_plot(x, labels, predictions, hidden_layers, clusters)

    sig.sankey(labels, keep, best_k, clusters, predictions)

In [3]:
keep = [1, 3 , 7]
n = 100
#interact(signature, n=widgets.IntSlider(min=50, max=300, step=10, value=100), keep=fixed(keep))
signature(n=50, keep=[3,7])

interactive(children=(IntSlider(value=100, description='n', max=300, min=50, step=10), Output()), _dom_classes…

<function __main__.signature(keep, n)>